In [1]:
import requests
import json
import pandas as pd
import demjson
from jsonpath import jsonpath

In [2]:
def get_data(year,month):
    url = 'http://api.tianapi.com/txapi/jiejiari/index?key=api密钥&date=' + year + '-' + month + '&mode=1&type=2'
    res = requests.get(url)
    json_data = demjson.decode(res.text)
    date_list = jsonpath(json_data, '$..date')
    info_list = jsonpath(json_data, '$..info')
    cnweekday_list = jsonpath(json_data, '$..cnweekday')
    name_list = jsonpath(json_data, '$..name')
    isnotwork_list = jsonpath(json_data, '$..isnotwork')
    vacation_list = jsonpath(json_data, '$..vacation')
    remark_list = jsonpath(json_data, '$..remark')
    tip_list = jsonpath(json_data, '$..tip')
    list_content = [date_list,info_list,cnweekday_list,name_list,isnotwork_list,vacation_list,remark_list,tip_list]
    df_jsonpath = pd.DataFrame(list_content)
    df_jsonpath = df_jsonpath.T
    df_jsonpath.rename(columns={
                                0:'公历日期',
                                1:'日期类型',
                                2:'公历星期',
                                3:'节假日名称',
                                4:'是否需要上班',
                                5:'节假日数组',
                                6:'调休日数组',
                                7:'放假提示',
                                },inplace=True)
    df_jsonpath.sort_values(by=['公历日期'],ascending=True,inplace=True)
    num_to_str = { 0 : '工作日', 
                   1 : '休息日', 
                  }
    df_jsonpath['是否需要上班'] = df_jsonpath['是否需要上班'].map(num_to_str)
    return df_jsonpath

In [3]:
year = '2021'
months = [str(i).rjust(2,'0') for i in range(1,13)]
data = pd.DataFrame()
for month in months:
    df = get_data(year,month)
    data = data.append(df)
print(data)

          公历日期 日期类型 公历星期     节假日名称 是否需要上班  \
0   2021-01-01  节假日  星期五       元旦节    休息日   
1   2021-01-02  节假日  星期六       元旦节    休息日   
2   2021-01-03  节假日  星期日       元旦节    休息日   
3   2021-01-04  工作日  星期一              工作日   
4   2021-01-05  工作日  星期二              工作日   
..         ...  ...  ...       ...    ...   
26  2021-12-27  工作日  星期一              工作日   
27  2021-12-28  工作日  星期二              工作日   
28  2021-12-29   节日  星期三  国际生物多样性日    工作日   
29  2021-12-30  工作日  星期四              工作日   
30  2021-12-31  工作日  星期五              工作日   

                                   节假日数组 调休日数组         放假提示  
0   [2021-01-01, 2021-01-02, 2021-01-03]        1月1日放假，共3天。  
1   [2021-01-01, 2021-01-02, 2021-01-03]        1月1日放假，共3天。  
2   [2021-01-01, 2021-01-02, 2021-01-03]        1月1日放假，共3天。  
3                                                            
4                                                            
..                                   ...   ...          ...  
26                       

In [4]:
filename = f'./{year}年节假日（自用版）.xlsx'
data.to_excel(filename,index=False)
print(f'生成 {filename} 成功！！')

生成 ./2021年节假日（自用版）.xlsx 成功！！
